In [ ]:
import numpy as np

class GreyWolfOptimizer:
    def __init__(self, func, lb, ub, dim, n_wolves, max_iter):
        self.func = func
        self.lb = lb
        self.ub = ub
        self.dim = dim
        self.n_wolves = n_wolves
        self.max_iter = max_iter
        self.positions = np.random.uniform(self.lb, self.ub, (self.n_wolves, self.dim))
        self.alpha_pos = np.zeros(self.dim)
        self.alpha_score = float("inf")
        self.beta_pos = np.zeros(self.dim)
        self.beta_score = float("inf")
        self.delta_pos = np.zeros(self.dim)
        self.delta_score = float("inf")

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.n_wolves):
                fitness = self.func(self.positions[i])
                if fitness < self.alpha_score:
                    self.alpha_score = fitness
                    self.alpha_pos = self.positions[i].copy()
                elif fitness < self.beta_score:
                    self.beta_score = fitness
                    self.beta_pos = self.positions[i].copy()
                elif fitness < self.delta_score:
                    self.delta_score = fitness
                    self.delta_pos = self.positions[i].copy()

            a = 2 - t * (2 / self.max_iter)
            for i in range(self.n_wolves):
                for j in range(self.dim):
                    r1 = np.random.random()
                    r2 = np.random.random()

                    A1 = 2 * a * r1 - a
                    C1 = 2 * r2
                    D_alpha = abs(C1 * self.alpha_pos[j] - self.positions[i, j])
                    X1 = self.alpha_pos[j] - A1 * D_alpha

                    r1 = np.random.random()
                    r2 = np.random.random()

                    A2 = 2 * a * r1 - a
                    C2 = 2 * r2
                    D_beta = abs(C2 * self.beta_pos[j] - self.positions[i, j])
                    X2 = self.beta_pos[j] - A2 * D_beta

                    r1 = np.random.random()
                    r2 = np.random.random()

                    A3 = 2 * a * r1 - a
                    C3 = 2 * r2
                    D_delta = abs(C3 * self.delta_pos[j] - self.positions[i, j])
                    X3 = self.delta_pos[j] - A3 * D_delta

                    self.positions[i, j] = (X1 + X2 + X3) / 3

                self.positions[i] = np.clip(self.positions[i], self.lb, self.ub)

        return self.alpha_pos, self.alpha_score
# Objective function and constraints
def objective_function(x):
    x1, x2, x3, x4, x5 = x
    return np.exp(x1 * x2 * x3 * x4 * x5) - 0.5 * (x1**3 + x2**3 + 1)**2

def constraint1(x):
    x1, x2, x3, x4, x5 = x
    return x1**2 + x2**2 + x3**2 + x4**2 + x5**2 - 10

def constraint2(x):
    x1, x2, x3, x4, x5 = x
    return x2 * x3 - 5 * x4 * x5

def constraint3(x):
    x1, x2, x3, x4, x5 = x
    return x1**3 + x2**3 + 1
def create_penalty_function(f,ineq_constraints=[(lambda x:0)],eq_constraints=[(lambda x:0)],rk=1):
    return lambda x:(f(x) +rk*sum([max(0,constraint(x))**2 for constraint in ineq_constraints])+sum([rk*constraint(x)**2 for constraint in eq_constraints]))
# Main execution
if __name__ == "__main__":
    n_vars = 5
    lower_bound = -5
    upper_bound = 5
    optim_points = []

    for i in range(20):
        print(f"Penalty function {i}:")
        penalty_function = create_penalty_function(objective_function, eq_constraints=[constraint1, constraint2,constraint3], rk=10**i)
        gwo_optimizer = GreyWolfOptimizer(penalty_function, lower_bound, upper_bound, n_vars, n_wolves=30, max_iter=100)
        
        # Perform optimization
        best_solution, best_fitness = gwo_optimizer.optimize()
        print(penalty_function(best_solution))
        optim_points.append(best_solution)


In [ ]:
for i in range(20):
    print(f"({objective_function(optim_points[i])},{constraint1(optim_points[i])},{constraint2(optim_points[i])},{constraint3(optim_points[i])})")

In [ ]:
class DifferentialEvolution:
    def __init__(self, func, lb, ub, dim, pop_size, max_iter, F=0.8, CR=0.9):
        self.func = func
        self.lb = lb
        self.ub = ub
        self.dim = dim
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.F = F
        self.CR = CR
        self.population = np.random.uniform(self.lb, self.ub, (self.pop_size, self.dim))
        self.scores = np.array([float("inf")] * self.pop_size)

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.pop_size):
                idxs = [idx for idx in range(self.pop_size) if idx != i]
                a, b, c = self.population[np.random.choice(idxs, 3, replace=False)]
                mutant = np.clip(a + self.F * (b - c), self.lb, self.ub)
                cross_points = np.random.rand(self.dim) < self.CR
                if not np.any(cross_points):
                    cross_points[np.random.randint(0, self.dim)] = True
                trial = np.where(cross_points, mutant, self.population[i])
                trial_score = self.func(trial)
                if trial_score < self.scores[i]:
                    self.scores[i] = trial_score
                    self.population[i] = trial

        best_idx = np.argmin(self.scores)
        return self.population[best_idx], self.scores[best_idx]

if __name__ == "__main__":
    n_vars = 5
    lower_bound = -10
    upper_bound = 10
    optim_points = []

    for i in range(20):
        print(f"Penalty function {i}:")
        penalty_function = create_penalty_function(objective_function, ineq_constraints=[constraint1, constraint2], eq_constraints=[constraint3], rk=10**i)
        de_optimizer = DifferentialEvolution(penalty_function, lower_bound, upper_bound, n_vars, pop_size=30, max_iter=100)
        
        # Perform optimization
        best_solution, best_fitness = de_optimizer.optimize()
        print(f"Best Fitness: {best_fitness}")
        optim_points.append(best_solution)

    print(optim_points)

In [ ]:
import numpy as np

class ParticleSwarmOptimizer:
    def __init__(self, func, lb, ub, dim, swarm_size, max_iter, w=0.5, c1=2, c2=2):
        self.func = func
        self.lb = lb
        self.ub = ub
        self.dim = dim
        self.swarm_size = swarm_size
        self.max_iter = max_iter
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.positions = np.random.uniform(self.lb, self.ub, (self.swarm_size, self.dim))
        self.velocities = np.random.uniform(-1, 1, (self.swarm_size, self.dim))
        self.p_best_positions = self.positions.copy()
        self.p_best_scores = np.array([float("inf")] * self.swarm_size)
        self.g_best_position = None
        self.g_best_score = float("inf")

    def optimize(self):
        for t in range(self.max_iter):
            for i in range(self.swarm_size):
                fitness = self.func(self.positions[i])
                if fitness < self.p_best_scores[i]:
                    self.p_best_scores[i] = fitness
                    self.p_best_positions[i] = self.positions[i].copy()
                if fitness < self.g_best_score:
                    self.g_best_score = fitness
                    self.g_best_position = self.positions[i].copy()

            for i in range(self.swarm_size):
                r1 = np.random.random(self.dim)
                r2 = np.random.random(self.dim)
                self.velocities[i] = (self.w * self.velocities[i] +
                                      self.c1 * r1 * (self.p_best_positions[i] - self.positions[i]) +
                                      self.c2 * r2 * (self.g_best_position - self.positions[i]))
                self.positions[i] = self.positions[i] + self.velocities[i]
                self.positions[i] = np.clip(self.positions[i], self.lb, self.ub)

        return self.g_best_position, self.g_best_score

# Objective function and constraints
def objective_function(x):
    x1, x2, x3, x4, x5 = x
    return np.exp(x1 * x2 * x3 * x4 * x5) - 0.5 * (x1**3 + x2**3 + 1)**2

def constraint1(x):
    x1, x2, x3, x4, x5 = x
    return x1**2 + x2**2 + x3**2 + x4**2 + x5**2 - 10

def constraint2(x):
    x1, x2, x3, x4, x5 = x
    return x2 * x3 - 5 * x4 * x5

def constraint3(x):
    x1, x2, x3, x4, x5 = x
    return x1**3 + x2**3 + 1

def create_penalty_function(f, ineq_constraints=[(lambda x: 0)], eq_constraints=[(lambda x: 0)], rk=1):
    return lambda x: (f(x) + rk * sum([max(0, constraint(x))**2 for constraint in ineq_constraints]) + sum([rk * constraint(x)**2 for constraint in eq_constraints]))

if __name__ == "__main__":
    # Setup PSO optimizer with problem parameters
    n_vars = 5
    lower_bound = -5
    upper_bound = 5
    optim_points = []

    for i in range(20):
        print(f"Penalty function {i}:")
        penalty_function = create_penalty_function(objective_function, ineq_constraints=[constraint1, constraint2], eq_constraints=[constraint3], rk=10**i)
        pso_optimizer = ParticleSwarmOptimizer(penalty_function, lower_bound, upper_bound, n_vars, swarm_size=30, max_iter=100)
        
        # Perform optimization
        best_solution, best_fitness = pso_optimizer.optimize()
        print(f"Best Fitness: {best_fitness}")
        optim_points.append(best_solution)

    print(optim_points)


In [ ]:
import numpy as np
from scipy.optimize import differential_evolution

# Set the bounds for each variable
bounds = [(-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5)]

# Perform Differential Evolution
for i in range(20):
    print(f"Penalty function {i}:")
    penalty_function = create_penalty_function(objective_function, 
                                               ineq_constraints=[constraint1, constraint2], 
                                               eq_constraints=[constraint3], rk=2**i)
    result = differential_evolution(create_penalty_function(objective_function,eq_constraints=[constraint1,constraint2,constraint3]), bounds, strategy='best1bin', maxiter=100, popsize=50)
    print("Best solution:", result.x)
    print("Best objective value:", result.fun)
        
    print(penalty_function(result.x))
    optim_points.append(result.x)
    print("Best position:", result.x)
    print("Best objective value:",  result.fun)

In [ ]:
import numpy as np
from scipy.optimize import basinhopping
initial_guess = np.array([-1.8,1.7,1.9,-0.8,-0.8])
for i in range(20):
    print(f"Penalty function {i}:")
    penalty_function = create_penalty_function(objective_function, 
                                               ineq_constraints=[constraint1, constraint2], 
                                               eq_constraints=[constraint3], rk=2**i)
    result = basinhopping(create_penalty_function(objective_function,eq_constraints=[constraint1,constraint2,constraint3]), initial_guess, niter=100, minimizer_kwargs={"method": "L-BFGS-B", "bounds": [(-5, 5)]*5})
    print("Best solution:", result.x)
    print("Best objective value:", result.fun)
        
    print(penalty_function(result.x))
    optim_points.append(result.x)
    print("Best position:", result.x)
    print("Best objective value:",  result.fun)


In [ ]:
import numpy as np
from scipy.optimize import dual_annealing

# Define the objective function
def objective_function(x):
    x1, x2, x3, x4, x5 = x
    return np.exp(x1 * x2 * x3 * x4 * x5) - 0.5 * (x1**3 + x2**3 + 1)**2

# Define the constraints
def constraint1(x):
    x1, x2, x3, x4, x5 = x
    return x1**2 + x2**2 + x3**2 + x4**2 + x5**2 - 10

def constraint2(x):
    x1, x2, x3, x4, x5 = x
    return x2 * x3 - 5 * x4 * x5

def constraint3(x):
    x1, x2, x3, x4, x5 = x
    return x1**3 + x2**3 + 1

# Combine the constraints into a single function
def constraints(x):
    return [constraint1(x), constraint2(x), constraint3(x)]

# Define the penalty function for the constraints
def penalty_function(x, rk):
    penalty = 0
    for constraint in constraints(x):
        penalty += rk * abs(constraint)  # Adding absolute value of constraints as penalty
    return penalty

# Define the objective function with penalty
def create_penalty_function(objective_function, ineq_constraints, eq_constraints, rk):
    def penalty_function(x):
        penalty = 0
        for constraint in ineq_constraints:
            if constraint(x) > 0:
                penalty += rk * constraint(x)
        for constraint in eq_constraints:
            penalty += rk * abs(constraint(x))
        return objective_function(x) + penalty
    return penalty_function

# Set the bounds for each variable
lower_bound = [-5, -5, -5, -5, -5]
upper_bound = [5, 5, 5, 5, 5]
bounds = list(zip(lower_bound, upper_bound))

# Perform Dual Annealing
optim_points = []

for i in range(5):
    print(f"Penalty function {i}:")
    penalty_function = create_penalty_function(objective_function, 
                                               ineq_constraints=[constraint1, constraint2], 
                                               eq_constraints=[constraint3], rk=10**i)
    result = dual_annealing(penalty_function, bounds)
    
    print(penalty_function(result.x))
    optim_points.append(result.x)




In [ ]:
import numpy as np
from pyswarm import pso


# Bounds for each variable
lb = [-5, -5, -5, -5, -5]
ub = [5, 5, 5, 5, 5]

# Perform PSO

for i in range(20):
    print(f"Penalty function {i}:")
    penalty_function = create_penalty_function(objective_function, 
                                               ineq_constraints=[constraint1, constraint2], 
                                               eq_constraints=[constraint3], rk=1.1**i)
    xopt, fopt = pso(penalty_function, lb, ub, swarmsize=50, maxiter=1000)
    
    print(penalty_function(xopt))
    optim_points.append(xopt)
    print("Best position:", xopt)
    print("Best objective value:", fopt)
